<a href="https://colab.research.google.com/github/mainarel/Numerical_methods_7/blob/main/Lab5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Решить численно разностным методом и методом конечных элементов с кусочно-линейными базисными функциями граничную задачу. Сравнить результаты решения полученными двумя методами с точным решением.*

$$ 
\frac{d^2 u}{dx^2} =f(x),\\ 0<x<1, u(0) = 0, u(1) = 0
$$

Вариант 3. 
$$
f(x) = x^3 
$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
xmin = 0
xmax = 1
hx = 0.01
    

nx = int((xmax - xmin) / hx) +1
x_array, hx = np.linspace(xmin,xmax, nx, retstep = True)



In [3]:
def f(x):
    return np.x ** 3

def solution(x):
    return 19/20 * np.x + (np.x ** 5) / 20


def basis():
    basis_ = [value_v_k(k) for k in range (1, nx-1)]
    basis_ = np.array(basis)
    return None

def value_v_k(k):
        def v_k(x):
            return np.piecewise(x, [x_array[k-1] <= x <= x_array[k], x_array[k] < x <= x_array[k+1], x < x_array[k-1] or x > x_array[k+1]],
                [lambda x: (x - x_array[k-1])/(x_array[k] - x_array[k-1]), 
                 lambda x: (x_array[k+1] - x)/(x_array[k+1] - x_array[k]),
                 lambda x :0])
        return v_k


def matrix_phi():
    phi = []
    for i in range(1, nx-1):
            phi.append([])
            for j in range(1, nx-1):
                if i == j:
                    I = 2/hx
                elif (i == j + 1) or (i == j - 1):
                    I = -1/hx
                else:
                    I = 0
                phi[i-1].append(I)
    phi = np.array(phi)
    return None


